In [1]:
import pandas
import json
import multiprocessing

1


In [2]:
path_to_file = r'D:\s_a\All_Amazon_Review.json'

In [8]:
# Slice large file to plenty of smaller ones

out = []
output_lines = 1200000
output_file_path = r'D:\s_a\file_{}'
i = 0
with open(path_to_file) as file:
    for line in file:
        out.append(line)
        if len(out) > output_lines:
            print(i)
            with open(output_file_path.format(i), mode='a') as file2:
                file2.write(json.dumps(out))
                i+=1
                out = []
    if len(out) != 0:
        with open(output_file_path.format(i), mode='a') as file2:
                file2.write(json.dumps(out))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193


In [ ]:
# Preprocess data

import pandas as pd
import json
import multiprocessing

# Number of files
files_in_folder = 194
cores = 6
input_file_path = r'D:\s_a\file_{}'

files_name = [input_file_path.format(x) for x in range(files_in_folder + 1)]


def json_to_csv(file_name):
    print(file_name)
    # Process data
    # Please notice that we using same name of variable to decrease memory usage

    with open(file_name) as f:
        file = json.load(f)

    # Unpack json rows
    file = [json.loads(x) for x in file]

    file = pd.DataFrame(file)

    # Remvove unnecessary columns
    file = file.filter(['overall', 'reviewText'])

    # Rename columns
    file = file.rename(columns={'overall': 'sentiment', 'reviewText': 'text'})

    # Transorm 0-5 score, into positive/negative values
    file.sentiment[file.sentiment.isin([4, 5])] = 'positive'
    file.sentiment[file.sentiment.isin([0, 1, 2])] = 'negative'

    # Remove values equal to 3
    file = file[(file.sentiment != 3)]

    # Write into csv file
    file.to_csv(file_name, mode='w', index=False)


if __name__ == '__main__':
    with multiprocessing.Pool(cores) as p:
        p.map(json_to_csv, files_name)

In [5]:
# Resize files

import pandas as pd

files_in_folder = 194
new_file_size = 8000000
input_file_path = r'D:\s_a\file_{}'
output_file_name = r'D:\s_a\file_large_{}'
output_index = 0

list_files_1 = [x for x in range(files_in_folder)]
list_files_2 = [x for x in range(files_in_folder)]
df2 = pd.DataFrame()

for i in list_files_1:
    print(i)
    list_files_2.remove(i)
    df1 = pd.read_csv(input_file_path.format(i))

    # Get size of dataframes
    length_df1 = df1.shape[0]
    length_df2 = df2.shape[0]

    if length_df2 + length_df1 > new_file_size:
        # Add missing values to data frame to fill in the missing
        df2 = pd.concat([df1[:new_file_size - length_df2], df2])
        df1 = df1[new_file_size - length_df2:]
        df2.to_csv(output_file_name.format(output_index), mode='a', index=False)
        output_index += 1
        df2 = pd.concat([df1])
    else:
        df2 = pd.concat([df1, df2])


In [ ]:
import pandas as pd
import time

# Drop duplicates
input_file_name = r'D:\s_a\file_large_{}'
files_in_folder = 26
new_file_size = 4000000
list_files_1 = [x for x in range(files_in_folder)]
list_files_2 = [x for x in range(files_in_folder)]

tt = time.time()
for i in list_files_1:
    print(i)
    list_files_2.remove(i)
    df1 = pd.read_csv(input_file_name.format(i))
    # Remove duplicated rows in data frame
    df1 = df1.drop_duplicates()
    
    # Prepare positive list of sentences and filter them 
    pos1 = set(df1[df1.sentiment == 'positive']['text'])
    pos1 = set(filter(lambda x: type(x) == str, pos1))
    
    # Prepare negative list of sentences and filter them 
    neg1 = set(df1[df1.sentiment == 'negative']['text'])
    neg1 = set(filter(lambda x: type(x) == str, neg1))
    
    for i2 in list_files_2:
        print('\t{}'.format(i2))
        df2 = pd.read_csv(input_file_name.format(i2))
        df2 = df2.drop_duplicates()
        pos2 = set(df2[df2.sentiment == 'positive']['text'])
        neg2 = set(df2[df2.sentiment == 'negative']['text'])
        
        # Remove duplicates sentences 
        pos1 = pos1 - pos2
        neg1 = neg1 - neg2

    pos1 = [{'sentiment': 'positive', 'text': x} for x in pos1]
    neg1 = [{'sentiment': 'negative', 'text': x} for x in neg1]
    df1 = pd.DataFrame(pos1 + neg1)
    df1.to_csv(input_file_name.format(i), mode='w', index=False)

print(time.time()-tt)
# 4h!